In [ ]:
%run -i rename.py

In [ ]:
from torchvision.io import read_image, write_jpeg, read_video, write_video
#filename = '../upload/v/{}.png'
#video = torch.stack([read_image(filename.format(str(i).zfill(8))) for i in range(62)])
video, audio, metadata = read_video('realshort.mp4')
video.shape, video.dtype

In [ ]:
from videoSR import doVSR, modules, ramCoef
from runSlomo import getOptP, getOptS
from progress import Node
modelPath = '../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth'
opt = getOptP(getOptS(modelPath, modules, ramCoef))
opt.start = 3
opt.end = -3
identity = lambda x=None, *_: x
p = doVSR(identity, Node({'IconVSR': 'VSR'}, learn=0), opt)
from imageProcess import extend
val_range = 2.0 ** 8
inp = video.permute(0, 3, 1, 2).to(dtype=config.dtype(), device=config.device()) / val_range


In [ ]:
res = []
for frame in inp:
  extend(res, p(frame))
extend(res, p(None))
x = torch.stack(res)
output = (x * val_range).clamp_(0, val_range - 1)
video = output.to(dtype=torch.uint8, device=torch.device('cpu')).permute(0, 2, 3, 1)
video.shape, x.mean(), x.abs().max(), (x > 1).sum() / x.nelement()

In [ ]:
from basicvsr_arch import IconVSR
modelPath = './IconVSR_Vimeo90K_BDx4-cfcb7e00.pth'
m = IconVSR(keyframe_stride=7, temporal_padding=3, num_block=30)
m.load_state_dict(torch.load(modelPath, map_location='cpu')['params'])
for param in m.parameters():
  param.requires_grad_(False)
m.eval()
m = m.to(dtype=config.dtype(), device=config.device())
val_range = 2.0 ** 8
inp = video.permute(0, 3, 1, 2).unsqueeze(0).to(dtype=config.dtype(), device=config.device()) / val_range


In [ ]:
x = m(inp)
output = (x * val_range).clamp_(0, val_range - 1)
video = output.to(dtype=torch.uint8, device=torch.device('cpu')).squeeze(0).permute(0, 2, 3, 1)
video.shape, x.mean(), x.abs().max(), (x > 1).sum() / x.nelement()

In [ ]:
def normalize(x, centralize=False, normalize=False, val_range=255.0):
    if centralize:
        x = x - val_range / 2
    if normalize:
        x = x / val_range

    return x


def normalize_reverse(x, centralize=False, normalize=False, val_range=255.0):
    if normalize:
        x = x * val_range
    if centralize:
        x = x + val_range / 2

    return x.clamp(0, val_range - 1)
val_range = 2.0 ** 8
inp = video.to(dtype=config.dtype(), device=config.device())
inp = normalize(inp, centralize=para.centralize, normalize=para.normalize, val_range=val_range).unsqueeze(0)
output = normalize_reverse(m(inp).squeeze(0), centralize=para.centralize, normalize=para.normalize, val_range=val_range)
video = output.to(dtype=torch.uint8, device=torch.device('cpu'))
video.shape

In [ ]:
filename = '../download/target/{}.jpg'
for i, img in enumerate(video.permute(0, 3, 1, 2)):
  write_jpeg(img.to(dtype=torch.uint8), filename.format(str(i).zfill(2)), quality=98)

In [ ]:
write_video('../download/target.mp4',
  video_array=video, fps=metadata['video_fps'], audio_array=audio, audio_fps=metadata['audio_fps'], audio_codec='aac')